<a href="https://colab.research.google.com/github/mishra28soumya/TensorFlow-for-Deep-Learning/blob/master/DL%20POC%20Model%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import callbacks
from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras import utils as np_utils
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np

In [0]:
from datetime import datetime
from packaging import version

In [0]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [0]:
n_train = int(x_train.shape[0] * 0.10)

In [0]:
_x_train = np.array(x_train, copy=True)
_y_train = np.array(y_train, copy=True)
_x_test = np.array(x_test, copy=True)
_y_test = np.array(y_test, copy=True)

In [36]:
print("n_train",n_train)
print(x_train.shape)

n_train 5000
(50000, 32, 32, 3)


In [0]:

x_test = np.concatenate([x_test, x_train[n_train:]], axis=0)
y_test = np.concatenate([y_test, y_train[n_train:]], axis=0)
x_train = x_train[0:n_train]
y_train = y_train[0:n_train]

In [38]:
print(_x_train.shape,"=>", x_train.shape)
print(_y_train.shape,"=>", y_train.shape)
print(_x_test.shape,"=>", x_test.shape)
print(_y_test.shape,"=>", y_test.shape)

(50000, 32, 32, 3) => (5000, 32, 32, 3)
(50000, 1) => (5000, 1)
(10000, 32, 32, 3) => (55000, 32, 32, 3)
(10000, 1) => (55000, 1)


In [0]:
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [0]:
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [41]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_12 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)       

In [0]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [0]:
#training
batch_size = 64
n_epochs = 5

In [0]:
np.random.shuffle(x_train)

In [45]:
opt_rms = optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=x_train.shape[0] // batch_size,epochs=n_epochs, verbose=1,validation_data=(x_test,y_test))


Epoch 1/5
78/78 [==============================] - 160s 2s/step - loss: 3.6387 - accuracy: 0.1010 - val_loss: 3.8826 - val_accuracy: 0.0911
Epoch 2/5
78/78 [==============================] - 161s 2s/step - loss: 3.2519 - accuracy: 0.0995 - val_loss: 3.7119 - val_accuracy: 0.0872
Epoch 3/5
78/78 [==============================] - 159s 2s/step - loss: 3.1156 - accuracy: 0.1086 - val_loss: 4.4091 - val_accuracy: 0.0915
Epoch 4/5
78/78 [==============================] - 162s 2s/step - loss: 3.0111 - accuracy: 0.0952 - val_loss: 3.2506 - val_accuracy: 0.0596
Epoch 5/5
78/78 [==============================] - 160s 2s/step - loss: 2.9233 - accuracy: 0.1041 - val_loss: 3.1242 - val_accuracy: 0.1077
